In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as  sns
import skimage.io
import glob
import re # REEEEEE

import cellpose

from cellpose import models
from progress.bar import Bar

PATH = 'data/INS1_BF/*/*.tiff'

In [ ]:
files = glob.glob(PATH)
print(type(files))
print(len(files))

In [ ]:
model = models.Cellpose(gpu=True, model_type='cyto')

path_2k = 'data/INS1_BF/2000/*.tiff'
files_2k = glob.glob(path_2k)
print(files[0])

In [ ]:
# columns=['filename', 'max', 'min','mean', 'Q1', 'Q3']
# sub dirs
dirs = glob.glob('data/INS1_BF/')

In [ ]:
dirs/

In [ ]:
def gen_data(path, model):
    files = glob.glob(path+'/*.tiff')
    imgs = [skimage.io.imread(f)for f in files]
    masks, flows, styles, diams = model.eval(imgs, diameter=None, channels=[0,0],  do_3D=False)
    masks=np.array(masks) # convert to numpy array
    counts = np.zeros(len(masks)) # empty array to hold counts
    bar = Bar(f'Loading Data: {path}',max=len(masks)) # progress
    for i,m in enumerate(masks):
        counts[i] = m.max()
        bar.next()
    bar.finish()
    return (counts.min(), np.quantile(counts, 0.25), counts.mean(), np.quantile(counts, 0.75), counts.max())

In [ ]:
mins, q1, means, q3, maxs = [],[],[],[],[]
for i, d in enumerate(dirs):
    print(f'Getting data from {d}')
    data = gen_data(d, model)
    mins.append(data[0])
    q1.append(data[1])
    means.append(data[2])
    q3.append(data[3])
    maxs.append(data[4])
    data = None

count_data = {
    'filenames':dirs,
    'min':mins,
    'q1':q1,
    'mean':means,
    'q3':q3,
    'max':maxs
}

In [ ]:
df = pd.DataFrame.from_dict(count_data)